# Text Generation


**Information**

**Tech Concept Generator**: Utilize LLMs to identify potential application scenarios for the "Soft Robotic Walker" (SRW) considering various combinations of options (highlighted in red).



*** 
**Background information**

* Conceptualization of current study with the demonstrator area in livmats.


***
**Coding sources**



***
**Aim of the code template**

Utilize LLMs to identify potential application scenarios for the "Soft Robotic Walker" (SRW) considering various combinations of options (highlighted in red on the slide).

## Get API key(s)

In [1]:
import os
import sys

# Assuming 'src' is one level down (in the current directory or a subdirectory)
path_to_src = os.path.join('../..','src')  # Moves one level down to 'src' folder

# Add the path to sys.path
sys.path.append(path_to_src)

# Now you can import your API_key module
import API_key as key

# Tech Concept Generator

## set up table of all possible combinations:

In [2]:
import itertools
import pandas as pd

# Define lists for each variable
#> Different Research Questions
researchQuestions = [
    'Ergonomic Tool Design and Assessment: How do user comfort, fatigue, and movement efficiency vary when interacting with ergonomically designed tools in simulated task scenarios?',
    'Force Perception and Haptic Feedback Studies: How accurately can users perceive and differentiate forces and tactile feedback, and how does this perception influence task precision and performance in HRI?',
    'Stress and Cognitive Load Evaluation: How do varying levels of physical and cognitive demands affect user performance, stress levels, and decision-making during human-robot interaction in complex tasks?',
    'Intuitive Control and Feedback Interfaces: How effectively can laypersons adapt to and operate robotic systems using intuitive control and feedback interfaces, and what factors influence their performance and ease of use?',
    'Social Acceptance of Robotic Systems: How do familiarity, experience, and robot behavior influence user trust, acceptance, and perceived usability of robotic systems in interactive tasks?',
]
studyDesigns = [
    'Within-Subjects Design: Test each participant under multiple conditions, such as different levels of force feedback, task complexity, or robot responsiveness. This reduces individual variability.',
    'Between-Subjects Design: Assign participants to distinct groups to test the effect of different interaction modalities (e.g., with/without VR, various haptic feedback levels).'
]


# Generate Cartesian product
product = list(itertools.product(researchQuestions, studyDesigns))

# Convert to a pandas DataFrame
df = pd.DataFrame(product, columns=['researchQuestion', 'studyDesign'])

# Add an ID column based on row index
df['ID'] = df.index

# Move ID column to the front
df = df[['ID', 'researchQuestion', 'studyDesign']]

# Display the DataFrame
print(df.shape)
print(df)

(10, 3)
   ID                                   researchQuestion  \
0   0  Ergonomic Tool Design and Assessment: How do u...   
1   1  Ergonomic Tool Design and Assessment: How do u...   
2   2  Force Perception and Haptic Feedback Studies: ...   
3   3  Force Perception and Haptic Feedback Studies: ...   
4   4  Stress and Cognitive Load Evaluation: How do v...   
5   5  Stress and Cognitive Load Evaluation: How do v...   
6   6  Intuitive Control and Feedback Interfaces: How...   
7   7  Intuitive Control and Feedback Interfaces: How...   
8   8  Social Acceptance of Robotic Systems: How do f...   
9   9  Social Acceptance of Robotic Systems: How do f...   

                                         studyDesign  
0  Within-Subjects Design: Test each participant ...  
1  Between-Subjects Design: Assign participants t...  
2  Within-Subjects Design: Test each participant ...  
3  Between-Subjects Design: Assign participants t...  
4  Within-Subjects Design: Test each participant ...  
5

## set up prompt templates:

In [3]:
system_template = """
<Context>
You are an advanced AI designed to generate innovative technological application scenarios for a Human-Robot Interaction (HRI) system.
The system integrates multiple components to create an immersive and interactive experimental platform. The key components of the setup are as follows:

1. **End Effector**: The robotic arm is equipped with a 6D shaker, enabling precise motion and force application in six degrees of freedom (translation along X, Y, Z axes, and rotation about these axes: pitch, yaw, roll).
2. **Human Interface**: Participants interact with the system via an ergonomic hand grip designed to mimic the shape and functionality of a power tool handle.
3. **Force Sensor**: A force sensor is installed between the hand grip and the shaker to measure real-time forces exerted during interaction.
4. **Motion Tracking**: The Vicon system is used to capture precise movements of the participant and the setup, enabling detailed analysis of interaction dynamics.
5. **Immersive Feedback**: Participants wear a VR headset to receive visual cues and experience simulated environments that enhance task immersion.

Your task is to generate application scenarios for this HRI system based on a combination of a **Research Question (RQ)** and a **Study Design**. The RQ specifies the goal of the study, while the Study Design outlines how the study is structured. 

The Research Question is:
{researchQuestion}

The Study Designs is:
{studyDesign}

Think creatively and ensure the application scenarios leverage the unique features of this HRI system to address the selected Research Question. Ensure the proposed application aligns with the selected Study Design and integrates the system's components effectively.
</Context>
"""

user_template = """
<Task>
Generate one detailed application scenario for the described HRI system. The scenario should be specific, feasible, and clearly illustrate how the system's components are utilized in a real-world or innovative setting. 

Here is an example format for the scenario output:

**Application Scenario**: [Detailed explanation of how the components of the system are utilized together to address a particular task or challenge];

Ensure that the scenario reflects thoughtful integration of the system's components and highlights their unique capabilities. The scenario should be realistic or conceptually sound and reflect a practical or innovative alignment with the system's capabilities.
Only return the scenario output which must end with an ";" and nothing else.
</Task>
"""


# Example invocation values for characteristics
example_values = {
    "researchQuestion": 'Ergonomic Tool Design and Assessment: How do user comfort, fatigue, and movement efficiency vary when interacting with ergonomically designed tools in simulated task scenarios?',
    "studyDesign": 'Within-Subjects Design: Test each participant under multiple conditions, such as different levels of force feedback, task complexity, or robot responsiveness. This reduces individual variability.'
}

# Generate the output by invoking the templates
system_template_filled = system_template.format(**example_values)
user_template_filled = user_template

print("system_template_invoked:")
print(system_template_filled)
print("\nuser_template_invoked:")
print(user_template_filled)

system_template_invoked:

<Context>
You are an advanced AI designed to generate innovative technological application scenarios for a Human-Robot Interaction (HRI) system.
The system integrates multiple components to create an immersive and interactive experimental platform. The key components of the setup are as follows:

1. **End Effector**: The robotic arm is equipped with a 6D shaker, enabling precise motion and force application in six degrees of freedom (translation along X, Y, Z axes, and rotation about these axes: pitch, yaw, roll).
2. **Human Interface**: Participants interact with the system via an ergonomic hand grip designed to mimic the shape and functionality of a power tool handle.
3. **Force Sensor**: A force sensor is installed between the hand grip and the shaker to measure real-time forces exerted during interaction.
4. **Motion Tracking**: The Vicon system is used to capture precise movements of the participant and the setup, enabling detailed analysis of interaction

## call LLMs to generate ideas

In [4]:
import pandas as pd
import re
from huggingface_hub import InferenceClient

# Initialize the client (assuming the client and API key are already set up)
client = InferenceClient(headers={"X-use-cache": "false"}, token=key.hugging_api_key)

def get_application_scenario(system_template_invoked, user_template_invoked):
    chat_completion = client.chat_completion(
        messages=[
            {"role": "system", "content": system_template_invoked},
            {"role": "user", "content": user_template_invoked}
        ],
        model="meta-llama/Meta-Llama-3.1-70B-Instruct",
        temperature=0.6,
        stream=False,
        max_tokens=1400
    )
    return chat_completion

def process_application_scenario(message_content, row):
    # Use regex to find the application scenario
    match = re.search(r'\*\*Application Scenario\*\*: (.*?;)', message_content, re.DOTALL)
    application_scenario = message_content

    if not match:
        print('The provided message content does not contain a valid Application Scenario, the "message_content" is taken.')
        # raise ValueError("The provided message content does not contain a valid 'Application Scenario'."
        # application_scenario = match.group(1).strip()

    # Extract the application scenario
    

    # Convert to DataFrame format and include row information
    df_scenarios = pd.DataFrame({
        "application_scenario": [application_scenario],
        "ID": row['ID'],
        "researchQuestion": row['researchQuestion'],
        "studyDesign": row['studyDesign']
    })

    return df_scenarios

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
     # Print the current processing ID for tracking
     print(f"Processing Row {index}: ID {row['ID']}")
     
     # Set up the system and user templates
     system_template_invoked = system_template.format(
         researchQuestion=row['researchQuestion'],
         studyDesign=row['studyDesign']
     )
     
     user_template_invoked = user_template  # The user template is static as it refers to the task format
     
     # Get the application scenario from the LLM
     chat_completion = get_application_scenario(system_template_invoked, user_template_invoked)
     
     # Process the returned content into a DataFrame
     scenario_df = process_application_scenario(chat_completion.choices[0].message.content, row)
     
     # Combine the result into a master DataFrame
     if index == 0:
         combined_df = scenario_df
     else:
         combined_df = pd.concat([combined_df, scenario_df], ignore_index=True)

Processing Row 0: ID 0
Processing Row 1: ID 1
Processing Row 2: ID 2
Processing Row 3: ID 3
Processing Row 4: ID 4
Processing Row 5: ID 5
Processing Row 6: ID 6
Processing Row 7: ID 7
Processing Row 8: ID 8
Processing Row 9: ID 9


In [5]:
chat_completion.choices[0].message.content

'**Application Scenario**: "Robot-Assisted Assembly Training in Virtual Reality"; \n\nIn this application scenario, the HRI system is used to investigate the impact of robotic assistance, haptic feedback, and virtual reality on user trust, acceptance, and performance in an assembly task. Participants are assigned to one of three groups: a control group without robotic assistance, a group with robotic assistance but without haptic feedback, and a group with robotic assistance and haptic feedback. Each participant wears the VR headset and grasps the ergonomic hand grip, which is equipped with the force sensor to measure the forces exerted during interaction. The 6D shaker end effector is programmed to provide precise motion and force cues to assist the participant in the assembly task. The Vicon motion tracking system captures the movement of the participant and the robotic arm, allowing researchers to analyze the interaction dynamics and task performance.\n\nThe study consists of two ph

In [6]:
# Display or use the combined DataFrame
print("Combined DataFrame with Application Scenarios:")
combined_df

Combined DataFrame with Application Scenarios:


,application_scenario,ID,researchQuestion,studyDesign
0,"**Application Scenario**: ""Ergonomic Tool Desi...",0,Ergonomic Tool Design and Assessment: How do u...,Within-Subjects Design: Test each participant ...
1,"**Application Scenario**: ""Ergonomic Power Too...",1,Ergonomic Tool Design and Assessment: How do u...,Between-Subjects Design: Assign participants t...
2,"**Application Scenario**: ""Virtual Assembly Tr...",2,Force Perception and Haptic Feedback Studies: ...,Within-Subjects Design: Test each participant ...
3,"**Application Scenario**: ""Haptic Training Sim...",3,Force Perception and Haptic Feedback Studies: ...,Between-Subjects Design: Assign participants t...
4,"**Application Scenario**: ""Assembly Task with ...",4,Stress and Cognitive Load Evaluation: How do v...,Within-Subjects Design: Test each participant ...
5,**Application Scenario**: Virtual Reality-Base...,5,Stress and Cognitive Load Evaluation: How do v...,Between-Subjects Design: Assign participants t...
6,"**Application Scenario**: ""Intuitive Robotic A...",6,Intuitive Control and Feedback Interfaces: How...,Within-Subjects Design: Test each participant ...
7,"**Application Scenario**: ""VR-Assisted Robotic...",7,Intuitive Control and Feedback Interfaces: How...,Between-Subjects Design: Assign participants t...
8,"**Application Scenario**: ""Robotic Assembly Ta...",8,Social Acceptance of Robotic Systems: How do f...,Within-Subjects Design: Test each participant ...
9,"**Application Scenario**: ""Robot-Assisted Asse...",9,Social Acceptance of Robotic Systems: How do f...,Between-Subjects Design: Assign participants t...


In [7]:
combined_df["application_scenario"][5]

"**Application Scenario**: Virtual Reality-Based Assembly Task with Varying Haptic Feedback and Cognitive Loads; \n\nIn this scenario, the HRI system is used to investigate how different levels of haptic feedback and cognitive demands impact user performance, stress levels, and decision-making during a complex assembly task. Participants are assigned to one of four groups in a between-subjects design: (1) high haptic feedback with complex instructions, (2) low haptic feedback with complex instructions, (3) high haptic feedback with simple instructions, and (4) low haptic feedback with simple instructions. \n\nThe task involves assembling a virtual robotic arm within a simulated environment displayed through the VR headset. The virtual arm consists of multiple components that require precise alignment and connection. The 6D shaker integrated into the robotic arm provides varying levels of haptic feedback, allowing participants to feel resistance, texture, or vibrations during the assemb

In [8]:
combined_df.to_excel("applicationScenarios_IPEK.xlsx", index=False)

## The “Stuff” Strategy (priotize)

for within subject designs

In [9]:
# Filter the DataFrame to get rows with specific conditions
filtered_df = combined_df[combined_df['studyDesign'] == "Within-Subjects Design: Test each participant under multiple conditions, such as different levels of force feedback, task complexity, or robot responsiveness. This reduces individual variability."]

# Combine the application_scenario texts
scenario_list = " ".join(filtered_df['application_scenario'].tolist())

# Display the filtered DataFrame and the combined scenario text (optional)
print("Filtered DataFrame:")
filtered_df

Filtered DataFrame:


,application_scenario,ID,researchQuestion,studyDesign
0,"**Application Scenario**: ""Ergonomic Tool Desi...",0,Ergonomic Tool Design and Assessment: How do u...,Within-Subjects Design: Test each participant ...
2,"**Application Scenario**: ""Virtual Assembly Tr...",2,Force Perception and Haptic Feedback Studies: ...,Within-Subjects Design: Test each participant ...
4,"**Application Scenario**: ""Assembly Task with ...",4,Stress and Cognitive Load Evaluation: How do v...,Within-Subjects Design: Test each participant ...
6,"**Application Scenario**: ""Intuitive Robotic A...",6,Intuitive Control and Feedback Interfaces: How...,Within-Subjects Design: Test each participant ...
8,"**Application Scenario**: ""Robotic Assembly Ta...",8,Social Acceptance of Robotic Systems: How do f...,Within-Subjects Design: Test each participant ...


In [10]:
print("\nCombined Application Scenarios:")
print(scenario_list)


Combined Application Scenarios:
**Application Scenario**: "Ergonomic Tool Design Optimization for Simulated Assembly Tasks"; 

In this scenario, the HRI system is used to investigate how user comfort, fatigue, and movement efficiency vary when interacting with ergonomically designed tools in simulated assembly tasks. The study employs a within-subjects design, where each participant performs a series of assembly tasks under different conditions.

The experiment begins with participants wearing the VR headset, which provides an immersive simulated environment of an assembly workspace. They are then asked to hold the ergonomic hand grip, which is equipped with the force sensor, and use it to manipulate virtual objects in the simulated environment. The 6D shaker, attached to the robotic arm, provides precise motion and force application, simulating the sensations of assembling parts.

The Vicon motion tracking system captures the participant's movements and the motion of the robotic arm,

In [11]:
# System and user template setup
system_template = """
<Context>
You are a helpful AI designed to **summarize innovative technological application scenarios for a Human-Robot Interaction (HRI) system**.
The system integrates multiple components that enable immersive and interactive experimental platforms for HRI research. 
The components include:
1. **End Effector**: A robotic arm equipped with a 6D shaker for precise motion and force application.
2. **Human Interface**: An ergonomic hand grip designed to mimic a power tool handle for intuitive interaction.
3. **Force Sensor**: Measures real-time forces exerted during user interaction.
4. **Motion Tracking**: Uses the Vicon system to capture precise movements of participants and setup for interaction analysis.
5. **Immersive Feedback**: Employs VR headsets to simulate environments and provide visual cues, enhancing user immersion.

Your task is to summarize and prioritize application scenarios based on a list of detailed descriptions. 
The goal is to identify the most significant and recurring applications of the HRI system based on the provided scenarios.
</Context>
"""


user_template = f"""
<Task>
Provide a prioritized list of possible applications for the described HRI system based on the following application scenarios. 
Start with the most critical and frequently mentioned applications, highlighting recurring themes and innovative use cases.

{scenario_list}

Ensure the list is ordered by significance and includes a brief explanation of why each application is relevant or innovative. 
Focus on aligning the identified applications with the capabilities of the HRI system components.
</Task>
"""

In [12]:
import pandas as pd
from huggingface_hub import InferenceClient

# Initialize the client (assuming the client and API key are already set up)
client = InferenceClient(headers={"X-use-cache": "false"}, token=key.hugging_api_key)

# Invoke the chat completion
chat_completion = client.chat_completion(
    messages=[
        {"role": "system", "content": system_template},
        {"role": "user", "content": user_template}
    ],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0.6,
    stream=False,
    max_tokens=1400
)

# Print the response
print("\nSummary of Application Scenarios:")
print(chat_completion.choices[0].message.content)


Summary of Application Scenarios:
Based on the provided application scenarios, I've identified the most significant and recurring applications of the HRI system, prioritizing them according to their relevance and innovation. Here's the list:

1. **Virtual Assembly Training with Haptic Feedback**: This application is crucial for industries that require precise assembly tasks, such as manufacturing and aerospace. The HRI system's immersive feedback and haptic cues enable researchers to investigate how accurately participants perceive and differentiate forces and tactile feedback, ultimately enhancing the effectiveness of virtual assembly training programs.

2. **Ergonomic Tool Design Optimization for Simulated Assembly Tasks**: This application has significant implications for improving worker safety and efficiency in manufacturing and other industries. By analyzing the data collected from the HRI system, researchers can gain a deeper understanding of how ergonomic tool design affects u

In [13]:
# Assuming chat_completion has been defined and contains the LLM response
summary_content = chat_completion.choices[0].message.content

# Define the file path
file_path = 'summary_of_application_scenarios_IPEK.txt'

# Write the content to a file
with open(file_path, 'w') as file:
    file.write(summary_content)

# Print confirmation
print(f"Summary content saved to {file_path}")

Summary content saved to summary_of_application_scenarios_IPEK.txt


## The “Stuff” Strategy (similarities)

for within subject designs

In [14]:
system_template = """
<Context>
You are a helpful AI designed to analyze and **summarize similarities across multiple innovative technological application scenarios** for a Human-Robot Interaction (HRI) system.
The system integrates multiple components that enable immersive and interactive experimental platforms for HRI research. 
The components include:
1. **End Effector**: A robotic arm equipped with a 6D shaker for precise motion and force application.
2. **Human Interface**: An ergonomic hand grip designed to mimic a power tool handle for intuitive interaction.
3. **Force Sensor**: Measures real-time forces exerted during user interaction.
4. **Motion Tracking**: Uses the Vicon system to capture precise movements of participants and setup for interaction analysis.
5. **Immersive Feedback**: Employs VR headsets to simulate environments and provide visual cues, enhancing user immersion.

Your task is to analyze and identify recurring themes and shared features among the application scenarios. 
The goal is to create a clear and concise bullet list summarizing the commonalities across the different scenarios based on their descriptions.
</Context>
"""


user_template = f"""
<Task>
Analyze the following application scenarios for the described HRI system and provide a concise bullet list summarizing their similarities. 
The summary should focus on recurring themes, shared system components, and common use cases across the scenarios.

{scenario_list}

Ensure that the bullet points are specific and highlight shared aspects such as system functionality, participant interaction, experimental setups, or application goals.
</Task>
"""


In [15]:
import pandas as pd
from huggingface_hub import InferenceClient

# Initialize the client (assuming the client and API key are already set up)
client = InferenceClient(headers={"X-use-cache": "false"}, token=key.hugging_api_key)

# Invoke the chat completion
chat_completion = client.chat_completion(
    messages=[
        {"role": "system", "content": system_template},
        {"role": "user", "content": user_template}
    ],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0.6,
    stream=False,
    max_tokens=1400
)

# Print the response
print("\nSummary of Application Scenarios:")
print(chat_completion.choices[0].message.content)


Summary of Application Scenarios:
**Similarities Across Application Scenarios:**

*   **Immersive Feedback**: All scenarios utilize the VR headset to provide an immersive simulated environment, enhancing user engagement and interaction.
*   **Ergonomic Hand Grip and Force Sensor**: Each scenario employs the ergonomic hand grip equipped with a force sensor, allowing participants to interact with virtual objects and providing real-time force exertion data.
*   **6D Shaker and Robotic Arm**: The 6D shaker attached to the robotic arm is used in all scenarios to provide precise motion and force application, simulating real-world assembly tasks and physical constraints.
*   **Vicon Motion Tracking**: The Vicon motion tracking system is used in all scenarios to capture participant movements and robotic arm motion, enabling detailed analysis of interaction dynamics.
*   **Within-Subjects Design**: Most scenarios employ a within-subjects design, where participants perform tasks under multiple 

In [16]:
# Assuming chat_completion has been defined and contains the LLM response
summary_content = chat_completion.choices[0].message.content

# Define the file path
file_path = 'similarities_of_application_scenarios_IPEK.txt'

# Write the content to a file
with open(file_path, 'w') as file:
    file.write(summary_content)

# Print confirmation
print(f"Summary content saved to {file_path}")

Summary content saved to similarities_of_application_scenarios_IPEK.txt
